# New York CitiBike Data Science Challenge

Der Fahrradverleih CitiBike vermietet in New York über 12.000 Fahrräder an 750 
Verleihstationen. Somit ist CitiBike eine echte Alternative zu den herkömmlichen 
Transportmitteln, wie z.B. U-bahn oder Taxi. 

Nehme an, es wäre das Jahr 2018. Mit einem gültigen 24 Stunden (3 Tage) Pass bzw. einer 
jährlichen Mitgliedschaft können Kunden ein Fahrrad an einer Verleihstation abholen und an 
einer beliebigen Station wieder abgeben. CitiBike stellt die durch den Verleih gesammelten 
Daten der Öffentlichkeit zur Verfügung. Deine Aufgabe als Data Scientist ist es, CitiBike dabei zu
helfen diese Daten wertstiftend zu nutzen. 

In einem ersten Pilotprojekt sollst du hierfür die Daten analysieren und ein Modell bauen, mit 
welchem zwei Klassen von Nutzern identifiziert werden können; dies sind (i) customers (24 
Stunden/ 3 Tage Pass) und (ii) subscribers (jährliche Mitgliedschaft).

Eine Orientierungshilfe bei 
dieser Aufgabenstellung bieten dir die folgenen Arbeitsschritte:
 
1. Lade diese Daten von Citibike für das Jahr 2018 herunter. Mach dich mit dem Inhalt des 
Datensatzes vertraut und bereite ihn für weitere Analysen auf. 
2. Visualisiere die Daten; sei kreativ und überlege dir geeignete Darstellungsformen für 
deine Entdeckungen. Nutze die Visualisierungen auch in deiner Ergebnispräsentation um 
deine Argumente zu unterstützen.
3. Überlege dir geeignete Features (Merkmale) als Input für dein customer-subscriber-
Modell. Konstruiere gegebenenfalls neue Features um dein Modell zu verbessern. 
4. Verteste verschiedene Modelle bzw. Methoden zur Klassifikation der Kundentypen 
subscribers und customers. Evaluiere die Performance der unterschiedlichen Modelle 
und begründe eine Modellauswahl.
5. Skizziere mögliche Einsatzgebiete oder UseCases deiner Modelle
6. Skizziere für CitiBike Kooperationsmöglichkeiten mit einer Versicherung (und/oder 
umgekehrt). 
 


## 0) Imports

In [71]:
import numpy as np
import pandas as pd

## 1) Datenvorbereitung

In [72]:
# Rohdaten einlesen
data = pd.read_csv('data/JC-201806-citibike-tripdata.csv')

# Datentyp verbessern
for column in data.columns:
    data.rename(columns={column: column.replace(' ', '_')}, inplace=True)
data.starttime = pd.to_datetime(data.starttime)
data.stoptime = pd.to_datetime(data.stoptime)

# Rohdaten sichten
data

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,329,2018-06-01 07:07:00.299,2018-06-01 07:12:30.222,3183,Exchange Place,40.716247,-74.033459,3276,Marin Light Rail,40.714584,-74.042817,29513,Subscriber,1991,1
1,259,2018-06-01 08:24:43.849,2018-06-01 08:29:03.845,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,26221,Subscriber,1989,2
2,686,2018-06-01 09:17:49.573,2018-06-01 09:29:16.177,3183,Exchange Place,40.716247,-74.033459,3202,Newport PATH,40.727224,-74.033759,29266,Subscriber,1990,1
3,164,2018-06-01 09:44:00.402,2018-06-01 09:46:44.683,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,29472,Subscriber,1993,1
4,769,2018-06-01 09:59:28.780,2018-06-01 10:12:18.666,3183,Exchange Place,40.716247,-74.033459,3203,Hamilton Park,40.727596,-74.044247,33679,Subscriber,1977,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40932,1227,2018-06-30 20:46:55.750,2018-06-30 21:07:22.973,3681,Grand St,40.715178,-74.037683,3209,Brunswick St,40.724176,-74.050656,29197,Subscriber,1979,2
40933,1219,2018-06-30 20:47:02.761,2018-06-30 21:07:22.172,3681,Grand St,40.715178,-74.037683,3209,Brunswick St,40.724176,-74.050656,33633,Customer,1969,0
40934,1227,2018-06-30 20:47:03.125,2018-06-30 21:07:30.356,3681,Grand St,40.715178,-74.037683,3209,Brunswick St,40.724176,-74.050656,26297,Subscriber,1987,2
40935,252,2018-06-30 21:13:35.689,2018-06-30 21:17:48.569,3681,Grand St,40.715178,-74.037683,3267,Morris Canal,40.712419,-74.038526,29482,Subscriber,1979,2


In [73]:
# Sanity checks der Rohdaten

print('Tripduration')
print(f'Min: {np.min(data.tripduration)} s, Max: {np.max(data.tripduration)/3600:.0f} h\n')

print('Starttime')
print(f'Min: {np.min(data.starttime)}, Max: {np.max(data.starttime)}\n')

print('Stoptime')
print(f'Min: {np.min(data.stoptime)}, Max: {np.max(data.stoptime)}\n')

print('Start station id')
print(f'Min: {np.min(data.start_station_id)}, Max: {np.max(data.start_station_id)}, #: {len(np.unique(data.start_station_id))}\n')

print('Start station name')
print(f'#: {len(np.unique(data.start_station_name))}\n')
      
print('Start station latitude')
print(f'Min: {np.min(data.start_station_latitude)}, Max: {np.max(data.start_station_latitude)}, #: {len(np.unique(data.start_station_latitude))}\n')

print('Start station longtude')
print(f'Min: {np.min(data.start_station_longitude)}, Max: {np.max(data.start_station_longitude)}, #: {len(np.unique(data.start_station_longitude))}\n')

print('End station id')
print(f'Min: {np.min(data.end_station_id)}, Max: {np.max(data.end_station_id)}, #: {len(np.unique(data.end_station_id))}\n')

print('End station name')
print(f'#: {len(np.unique(data.end_station_name))}\n')

print('End station latitude')
print(f'Min: {np.min(data.end_station_latitude)}, Max: {np.max(data.end_station_latitude)}, #: {len(np.unique(data.end_station_latitude))}\n')

print('End station longtude')
print(f'Min: {np.min(data.end_station_longitude)}, Max: {np.max(data.end_station_longitude)}, #: {len(np.unique(data.end_station_longitude))}\n')

print('Bikeid')
print(f'Min: {np.min(data.bikeid)}, Max: {np.max(data.bikeid)}, #: {len(np.unique(data.bikeid))}\n')

print('Usertype')
print(f'Types: {np.unique(data.usertype)}, #: {np.unique(data.usertype, return_counts=True)[1]}\n')

print('Birth year')
print(f'Min: {np.min(data.birth_year)}, Max: {np.max(data.birth_year)}\n')

print('Gender')
print(f'Types: {np.unique(data.gender)}, #: {np.unique(data.gender, return_counts=True)[1]}\n')

Tripduration
Min: 61 s, Max: 96 h

Starttime
Min: 2018-06-01 00:02:36.104000, Max: 2018-06-30 23:52:34.348000

Stoptime
Min: 2018-06-01 00:05:54.096000, Max: 2018-07-01 16:22:16.052000

Start station id
Min: 3183, Max: 3681, #: 50

Start station name
#: 50

Start station latitude
Min: 40.70965083364061, Max: 40.74871594538023, #: 50

Start station longtude
Min: -74.0836394, Max: -74.0321082, #: 50

End station id
Min: 157, Max: 3681, #: 63

End station name
#: 63

End station latitude
Min: 40.68763155, Max: 40.8143256, #: 63

End station longtude
Min: -74.0836394, Max: -73.9590255, #: 63

Bikeid
Min: 15582, Max: 33695, #: 621

Usertype
Types: ['Customer' 'Subscriber'], #: [ 3187 37750]

Birth year
Min: 1888, Max: 2002

Gender
Types: [0 1 2], #: [ 3105 28787  9045]



In [74]:
# Daten säubern

# Sortiere Fahrten aus, die länger als 12 Stunden dauerten
data = data[data.tripduration <= 12*3600]

# Macht es Sinn, Geburtsjahre vor 1918 auszusortieren?

## 2) Visualisierungen